# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   


In [9]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


Importación de librerías utilizadas en este archivo.

In [10]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm


A continuación, se indican el número de cubits del circuito a utilizar, la coordenada del ángulo polar (theta) y la coordenada del ángulo azimutal (phi).

Las expresiones para la obtención de armónicos esféricos se han obtenido para 4 cubits, por lo que es el circuito sobre le que se trabajará.

In [11]:
nCubits = 4
theta = math.pi/5
phi = math.pi/7

Ángulos utilizados para el cálculo de los armónicos esféricos. La primera coordenada corresponde con la coordenada azimutal y la segunda con la polar.

In [12]:
angles = [['Fc5.',-71,-21],
 ['Fc3.',-50,-28],
 ['Fc1.',-32,-45],
 ['Fcz.',23,90],
 ['Fc2.',32,45],
 ['Fc4.',50,28],
 ['Fc6.',71,21],
 ['C5..',-69,0],
 ['C3..',-46,0],
 ['C1..',23,0],
 ['Cz..',0,0],
 ['C2..',23,0],
 ['C4..',46,0],
 ['C6..',69,0],
 ['Cp5.',-71,21],
 ['Cp3.',-50,28],
 ['Cp1.',-32,45],
 ['Cpz.',23,-90],
 ['Cp2.',32,-45],
 ['Cp4.',50,-28],
 ['Cp6.',71,-21],
 ['Fp1.',-92,-72],
 ['Fpz.',92,90],
 ['Fp2.',92,72],
 ['Af7.',-92,-52],
 ['Af3.',-74,-67],
 ['Afz.',69,90],
 ['Af4.',74,67],
 ['Af8.',92,52],
 ['F7..',-92,-36],
 ['F5..',-75,-41],
 ['F3..',-60,-51],
 ['F1..',-50,-68],
 ['Fz..',46,90],
 ['F2..',50,68],
 ['F4..',60,51],
 ['F6..',75,41],
 ['F8..',92,36],
 ['Ft7.',-92,-18],
 ['Ft8.',92,18],
 ['T7..',-92,0],
 ['T8..',92,0],
 ['T9..',-115,0],
 ['T10.',115,0],
 ['Tp7.',-92,18],
 ['Tp8.',92,-18],
 ['P7..',-92,36],
 ['P5..',-75,41],
 ['P3..',-60,51],
 ['P1..',-50,68],
 ['Pz..',46,-90],
 ['P2..',50,-68],
 ['P4..',60,-51],
 ['P6..',75,-41],
 ['P8..',92,-36],
 ['Po7.',-92,54],
 ['Po3.',-74,67],
 ['Poz.',69,-90],
 ['Po4.',74,-67],
 ['Po8.',92,-54],
 ['O1..',-92,72],
 ['Oz..',92,-90],
 ['O2..',92,-72],
 ['Iz..',115,-90]
 ]

#Versión de los ángulos en radianes.
anglesRad = []
piFactor = math.pi/180
for x in angles:
    anglesRad.append([x[0], x[1] * piFactor, x[2] * piFactor])

In [28]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])


def EvalLegendre4(theta):
  nCubits = 4
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])

  #Polinomios de "m" negativo
  for l in range(1, ncubits + 1):
    for m in range(1, l+1):
      PolinomiosLegendre['P_' + str(l) + '_-' + str(m)] = ((-1)**(m) * (math.factorial(l-m)/math.factorial(l+m))) * PolinomiosLegendre['P_' + str(l) + '_' + str(m)]
  return PolinomiosLegendre

def EvalEsfericos4(theta, phi):
  l = 4
  legendre = EvalLegendre4(theta)
  AE = {}
  for k in range(l+1):
    for m in range(-k, k+1):
      coef =  math.sqrt(((2*k + 1)*math.factorial(k - (m))) / (4*math.pi * math.factorial(k+(m)))) #Coeficiente de normalización
      e = ExpPhi(phi, (m))
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(m)])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[0], legendre_normalizado * e[1])
  
  return AE

In [15]:
((-1)**1)*math.sqrt((((2*1 + 1)*math.factorial(1 - abs(1))) / (4*math.pi * math.factorial(1+abs(1)))))
math.sqrt(3/(8*math.pi))

0.3454941494713355

In [29]:
AE = EvalEsfericos4(theta, phi)
for armonicos in AE.values():
    print(armonicos[0], armonicos[1])

NameError: name 'ncubits' is not defined

In [17]:
m = [0,-1,0,1,-2, -1,0,1,2,-3,-2,-1,0,1,2,3,-4,-3,-2,-1,0,1,2,3,4]
v = [0, 1,1,1, 2, 2,2,2,2, 3, 3, 3,3,3,3,3, 4, 4, 4, 4,4,4,4,4,4]
x = []
for i in range(15):
    x.append(math.cos(theta))

sph = sph_harm(m, v, phi, theta)
sph

array([ 0.28209479+0.j        ,  0.18296548-0.08811153j,
        0.39528774+0.j        , -0.18296548-0.08811153j,
        0.08320749-0.10433889j,  0.33098767-0.15939526j,
        0.30388781+0.j        , -0.33098767-0.15939526j,
        0.08320749+0.10433889j,  0.01885368-0.08260335j,
        0.17810212-0.223333j  ,  0.38894232-0.18730475j,
        0.08228221+0.j        , -0.38894232-0.18730475j,
        0.17810212+0.223333j  , -0.01885368-0.08260335j,
       -0.01175414-0.05149827j,  0.04575883-0.20048254j,
        0.25808646-0.32363019j,  0.32056269-0.15437486j,
       -0.17369002+0.j        , -0.32056269-0.15437486j,
        0.25808646+0.32363019j, -0.04575883-0.20048254j,
       -0.01175414+0.05149827j])

In [18]:
for k in range(4+1):
    for m in range(-k, k+1):
        c = ((-1)**m)*math.sqrt((((2*k + 1)*math.factorial(k - abs(m))) / (4*math.pi * math.factorial(k+abs(m)))))
        print(c)

0.28209479177387814
-0.3454941494713355
0.4886025119029199
-0.3454941494713355
0.12875806734106318
-0.25751613468212636
0.6307831305050401
-0.25751613468212636
0.12875806734106318
-0.027814921575518937
0.06813236509555216
-0.21545345607610045
0.7463526651802308
-0.21545345607610045
0.06813236509555216
-0.027814921575518937
0.004214597070904597
-0.011920680675222404
0.044603102903819275
-0.18923493915151202
0.8462843753216345
-0.18923493915151202
0.044603102903819275
-0.011920680675222404
0.004214597070904597


In [19]:
ExpPhi(math.pi/7, -1)

(tensor(0.90096887, requires_grad=True),
 tensor(-0.43388374, requires_grad=True))

In [20]:
#math.sin(theta)**1
#math.cos(theta/2)**8 + 2*math.cos(theta/2)**6*math.sin(theta/2)**2 - 2*math.cos(theta/2)**2*math.sin(theta/2)**6  - math.sin(theta/2)**8
#A = [[a, -b],[b, 1]]
print(abs(-3))

3


In [21]:
cos = sp.symbols("cos")
sen = sp.symbols("sin")
cos2 = sp.symbols("cos2")
sen2 = sp.symbols("sin2")

C0 = cos**8
C1 = cos**6*sen**2
C3 = cos**4*sen**4
C7 = cos**2*sen**6
C15 = sen**8

S0 = cos2**8
S1 = cos2**6*sen2**2
S3 = cos2**4*sen2**4
S7 = cos2**2*sen2**6
S15 = sen2**8

X = (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[7] + 3)
X = (35*(C[0] - 4*C[1] + 6*C[3] - 4*C[7] + C[15]) - 30*(C[0] -2*C[3]  + C[15]) + 3)/8
X = (C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15])

X


NameError: name 'C' is not defined

In [22]:
legendre = EvalLegendre(theta)
legendre

{'P_0_0': 1,
 'P_1_0': tensor(0.80901699, requires_grad=True),
 'P_1_1': tensor(-0.58778525, requires_grad=True),
 'P_2_0': tensor(0.48176275, requires_grad=True),
 'P_2_1': tensor(-1.42658477, requires_grad=True),
 'P_2_2': tensor(1.03647451, requires_grad=True),
 'P_3_0': tensor(0.11024575, requires_grad=True),
 'P_3_1': tensor(-2.00365044, requires_grad=True),
 'P_3_2': tensor(4.19262746, requires_grad=True),
 'P_3_3': tensor(-3.04612215, requires_grad=True),
 'P_4_0': tensor(-0.20523836, requires_grad=True),
 'P_4_1': tensor(-1.88019056, requires_grad=True),
 'P_4_2': tensor(9.28048775, requires_grad=True),
 'P_4_3': tensor(-17.25055212, requires_grad=True),
 'P_4_4': tensor(12.53325974, requires_grad=True),
 'P_1_-1': tensor(0.29389263, requires_grad=True),
 'P_2_-1': tensor(0.23776413, requires_grad=True),
 'P_2_-2': tensor(0.04318644, requires_grad=True),
 'P_3_-1': tensor(0.16697087, requires_grad=True),
 'P_3_-2': tensor(0.03493856, requires_grad=True),
 'P_3_-3': tensor(0.004

In [23]:
k = 2
m = 1
legendreNegativo = ((-1)**(m) * (math.factorial(k-m)/math.factorial(k+m)))

legendre['P_' + str(k) + '_' + str(m)]  * legendreNegativo

tensor(0.23776413, requires_grad=True)

In [24]:
m = [-1,-1,-2,-1,-2,-3,-1,-2,-3,-4]
v = [ 1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
lp = lpmv(m, v, math.cos(theta))
lp

array([0.29389263, 0.23776413, 0.04318644, 0.16697087, 0.03493856,
       0.00423073, 0.09400953, 0.02577913, 0.00342273, 0.00031084])

In [25]:
m, k= sp.symbols('m, k')
#x = ((-1)**sp.Abs(m))*math.sqrt(((2*k + 1)*sp.factorial(k - sp.Abs(m))) / (4*sp.pi * sp.factorial(k+sp.Abs(m))))
x = ((-1)**sp.Abs(m))*sp.sqrt(((2*k + 1) * sp.factorial(k - sp.Abs(m))) / (4*sp.pi * sp.factorial(k + sp.Abs(m))))
x

(-1)**Abs(m)*sqrt((2*k + 1)*factorial(k - Abs(m))/factorial(k + Abs(m)))/(2*sqrt(pi))